In [1]:
import sys
sys.path.append('../..')
import src.data.data_loader as dl
from src.features.vectorizer import Vectorizer
from src.models.topic_models import TopicModel
import pandas as pd
import sklearn.utils as skutil
pd.set_option('display.max_rows', None)

C:\Users\Maria\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [2]:
# Data 
language = 'german'
typex = 'editorial'
aggregation = 'article'

# Vectorization
min_df = 0.005
max_df = 0.9

# Topic Modeling
algorithm = 'nmf'
num_topics = 170

In [3]:
data = dl.get_comments_by_type(language, typex, aggregation)
texts = data['comment_texts']

def min_length (texts,min_characters):
    neu = []
    for t in texts: 
        token_perdoc_list = t.split()
        token_min_character = []
        for token in token_perdoc_list: 
            if len(token)>= min_characters: 
                token_min_character.append(token)
        joined = (" ").join(token_min_character)
        neu.append(joined)
    return neu

texts = min_length(texts,3)

In [ ]:
vec = Vectorizer('tfidf', texts, min_df=min_df, max_df=max_df)
vec.save('tagged/vectorizer/{}_{}_{}_pos.pkl'.format(algorithm, language, typex,"tagged"))

document_term_matrix = vec.get_document_token_matrix(texts)
id2token = vec.get_id2token_mapping()
document_term_matrix = skutil.shuffle(document_term_matrix, random_state=1)
display(document_term_matrix.shape)

(1782, 15193)

In [ ]:
model = TopicModel('nmf',num_topics, document_term_matrix, id2token)
model.save('tagged/topic_models/nmf/{}_{}_{}_{}_comments_{}.pkl'.format(algorithm, language, typex,"tagged",num_topics))